In [1]:
# TEMP: Import lagom
# Not useful once lagom is installed
import sys
sys.path.append('/home/zuo/Code/lagom/')

In [2]:
from lagom.envs.vec_env import DummyVecEnv


### use functiontools.partial to set make_env function without argument but internally

In [5]:
import gym

from lagom.envs import GymEnv

def make_env():
    env = gym.make('Acrobot-v1')
    
    return GymEnv(env)

env = DummyVecEnv([make_env, make_env])
env.reset()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


array([[ 0.99818915,  0.06015327,  0.99840103, -0.05652779,  0.0641808 ,
         0.02995456],
       [ 0.99792521,  0.06438386,  0.99739941,  0.0720723 ,  0.08908737,
         0.01363955]])

In [4]:
import numpy as np
import gym


class DummyVecEnv(VecEnv):
    def __init__(self, env_fns):
        self.envs = [fn() for fn in env_fns]
        env = self.envs[0]
        super().__init__(len(env_fns), env.observation_space, env.action_space)

        obs_spaces = self.observation_space.spaces if isinstance(self.observation_space, gym.spaces.Tuple) else (self.observation_space,)
        self.buf_obs = [np.zeros((self.num_envs,) + tuple(s.shape), s.dtype) for s in obs_spaces]
        self.buf_dones = np.zeros((self.num_envs,), dtype=np.bool)
        self.buf_rews  = np.zeros((self.num_envs,), dtype=np.float32)
        self.buf_infos = [{} for _ in range(self.num_envs)]
        self.actions = None

    def step_async(self, actions):
        self.actions = actions

    def step_wait(self):
        for i in range(self.num_envs):
            obs_tuple, self.buf_rews[i], self.buf_dones[i], self.buf_infos[i] = self.envs[i].step(self.actions[i])
            if self.buf_dones[i]:
                obs_tuple = self.envs[i].reset()
            if isinstance(obs_tuple, (tuple, list)):
                for t,x in enumerate(obs_tuple):
                    self.buf_obs[t][i] = x
            else:
                self.buf_obs[0][i] = obs_tuple
        return (self._obs_from_buf(), np.copy(self.buf_rews), np.copy(self.buf_dones),
                self.buf_infos.copy())

    def reset(self):
        for i in range(self.num_envs):
            obs_tuple = self.envs[i].reset()
            if isinstance(obs_tuple, (tuple, list)):
                for t,x in enumerate(obs_tuple):
                    self.buf_obs[t][i] = x
            else:
                self.buf_obs[0][i] = obs_tuple
        return self._obs_from_buf()

    def close(self):
        return

    def _obs_from_buf(self):
        if len(self.buf_obs) == 1:
            return np.copy(self.buf_obs[0])
        else:
            return tuple(np.copy(x) for x in self.buf_obs)

In [ ]:
import numpy as np
from multiprocessing import Process  # easier code than threading
from multiprocessing import Pipe  # Much faster than Queue


class CloudpickleWrapper(object):
    """
    Uses cloudpickle to serialize contents (otherwise multiprocessing tries to use pickle)
    """
    def __init__(self, x):
        self.x = x
        
    def __call__(self):
        return self.x()
    
    def __getstate__(self):
        import cloudpickle
        return cloudpickle.dumps(self.x)
    def __setstate__(self, ob):
        import pickle
        self.x = pickle.loads(ob)


def worker(child_conn, parent_conn, make_env):
    parent_conn.close()
    # Create an environment
    env = make_env()
    
    while True:
        cmd, data = child_conn.recv()
        if cmd == 'step':
            obs, reward, done, info = env.step(data)
            if done:
                obs = env.reset()  # TODO: why reset
            child_conn.send([obs, reward, done, info])
        elif cmd == 'reset':
            obs = env.reset()
            child_conn.send(obs)
        elif cmd == 'reset_task':
            obs = env.reset_task()
            child_conn.send(obs)
        elif cmd == 'close':
            child_conn.close()
            break
        elif cmd == 'get_spaces':
            child_conn.send([env.observation_space, env.action_space])
        else:
            raise NotImplementedError

class SubprocVecEnv(VecEnv):
    """
    Run a list of environment in subprocesses
    """
    def __init__(self, list_make_env):
        self.waiting = False
        self.closed = False
        self.num_envs = len(list_make_env)
        self.parent_conns, self.child_conns = zip(*[Pipe() for _ in range(self.num_envs)])
        self.processes = []
        for parent_conn, child_conn, make_env in zip(self.parent_conns, self.child_conns, list_make_env):
            self.processes.append(Process(target=worker, args=[child_conn, parent_conn, CloudpickleWrapper(make_env)]))  # TODO: CloudpickleWrapper make_env
        for process in self.processes:
            process.daemon = True  # if the main process crashes, we should not cause things to hang
            process.start()
        for conn in self.child_conns:
            conn.close()
        
        # Obtain observation and action spaces
        self.parent_conns[0].send(['get_spaces', None])
        observation_space, action_space = self.parent_conns[0].recv()
        super().__init__(self.num_envs, observation_space, action_space)
        
    def step_asyn(self, actions):
        for parent_conn, action in zip(self.parent_conns, actions):
            parent_conn.send(['step', action])
            
        self.waiting = True
        
    def step_wait(self):
        observations, rewards, dones, infos = zip(*[parent_conn.recv() for parent_conn in self.parent_conns])
        self.waiting = False
        return np.stack(observations), np.stack(rewards), np.stack(dones), infos
    
    def reset(self):
        for parent_conn in self.parent_conns:
            parent_conn.send(['reset', None])
        return np.stack([parent_conn.recv() for parent_conn in self.parent_conns])
    
    def reset_task(self):
        for parent_conn in self.parent_conns:
            parent_conn.send(['reset_task', None])
        return np.stack([parent_conn.recv() for parent_conn in self.parent_conns])
    
    def close(self):
        if self.closed:
            return
        if self.waiting:
            for parent_conn in self.parent_conns:
                parent_conn.recv()
        for parent_conn in self.parent_conns:
            parent_conn.send(['close', None])
        for process in self.processes:
            process.join()
        self.closed = True

In [3]:
from lagom.core

In [53]:
import torch.nn.functional as F

conv = CNN(3, [128, 128], [4, 4, 4], [3, 3, 3], [3, 3, 3], [0, 0, 0], F.relu, [32, 16], F.relu, 2, None)

x = torch.zeros(20, 3, 128, 128)
print(conv)
conv(x).size()

CNN(
  (conv_layers): Sequential(
    (Conv 1): Conv2d(3, 4, kernel_size=(3, 3), stride=(3, 3))
    (Conv 2): Conv2d(4, 4, kernel_size=(3, 3), stride=(3, 3))
    (Conv 3): Conv2d(4, 4, kernel_size=(3, 3), stride=(3, 3))
  )
  (MLP): MLP(
    (hidden_layers): Sequential(
      (FC 1): Linear(in_features=64, out_features=32, bias=True)
      (FC 2): Linear(in_features=32, out_features=16, bias=True)
    )
    (output_layer): Linear(in_features=16, out_features=2, bias=True)
  )
)


torch.Size([20, 2])

In [45]:
import torch.nn as nn
import torch.nn.functional as F

from .MLP import MLP


class CNN(nn.Module):
    """
    Convolutional neural networks, from DeepMind Nature paper. 
    """
    def __init__(self, 
                 input_channel, 
                 input_shape,
                 conv_kernels,
                 conv_kernel_sizes,
                 conv_strides, 
                 conv_pads, 
                 conv_nonlinearity, 
                 hidden_sizes=None, 
                 hidden_nonlinearity=None, 
                 output_dim=None, 
                 output_nonlinearity=None):
        """
        Set up CNN with configurations. 
        
        Args:
            input_channel (int): the number of channels of the input, e.g. color channel
            input_shape (list): [Height, Width] of the input, to calculate the 
            conv_kernels (list): a list of number of kernels (filters or feature maps), for each convolutional layer. 
            conv_kernel_sizes (list): a list of kernel sizes, [int or tuple], for each convolutional layer. 
            conv_strides (list): a list of strides, for each convolutional layer. 
            conv_pads (list): a list of paddings, for each convolutional layer. 
            conv_nonlinearity (nn.functional): nonlinearity for convolutional layers
            hidden_sizes (list): a list of sizes for hidden layers
            hidden_nonlinearity (nn.functional): nonlinearity for hidden layers
            output_dim (int): output dimension
                                If None, then no output layer to be generated (useful if output has different heads)
            output_nonlinearity (nn.functional): nonlinearity for output layer
                                If None, then no output nonlinearity
        """
        super().__init__()
        
        self.input_channel = input_channel
        self.input_shape = input_shape
        self.conv_kernels = conv_kernels
        self.conv_kernel_sizes = conv_kernel_sizes
        self.conv_strides = conv_strides
        self.conv_pads = conv_pads 
        self.conv_nonlinearity = conv_nonlinearity
        self.hidden_sizes = hidden_sizes
        self.hidden_nonlinearity = hidden_nonlinearity
        self.output_dim = output_dim
        self.output_nonlinearity = output_nonlinearity
        
        # Iteratively build convolutional layers
        # Should use nn.Sequential, otherwise cannot be recognized
        self.conv_layers = nn.Sequential()
        # Augment the input channel to the list of conv_kernels
        kernels = [self.input_channel] + self.conv_kernels
        for i in range(len(self.conv_kernels)):
            in_channels = kernels[i]
            out_channels = kernels[i+1]
            conv = nn.Conv2d(in_channels=in_channels,
                             out_channels=out_channels,
                             kernel_size=self.conv_kernel_sizes[i], 
                             stride=self.conv_strides[i], 
                             padding=self.conv_pads[i])
            self.conv_layers.add_module(f'Conv {i+1}', conv)

        # MLP
        if self.hidden_sizes is not None:
            # Temporary forward pass to determine MLP input_dim
            input_dim = self._conv_out_dim()
            # Create MLP
            self.MLP = MLP(input_dim=input_dim,
                           hidden_sizes=self.hidden_sizes, 
                           hidden_nonlinearity=self.hidden_nonlinearity,
                           output_dim=self.output_dim, 
                           output_nonlinearity=self.output_nonlinearity)
        
        # Initialize parameters
        self._init_params()
        
    def forward(self, x):
        # Forward pass through convolutional layers
        x = self._conv_forward(x)
        
        if self.hidden_sizes is not None:
            # Flatten the tensors for MLP
            x = x.view(x.size(0), -1)
            
            # Forward pass through MLP
            x = self.MLP(x)
            
        return x
        
    def _conv_forward(self, x):
        for conv in self.conv_layers:
            x = self.conv_nonlinearity(conv(x))
            
        return x
    
    def _conv_out_dim(self):
        x = torch.zeros(1, self.input_channel, *self.input_shape)
        x = self._conv_forward(x)
        out_dim = x.view(1, -1).size(1)
        
        return out_dim
        
    def _init_params(self):
        """
        Initialize the parameters for convolutional layers, filter weights and biases
        
        Orthogonal weight initialization and zero bias initialization
        """
        # TODO: more flexible initialization API
        # Iterate over all convolutional layers
        for conv in self.conv_layers:
            # Calculate gain for the nonlinearity
            gain = nn.init.calculate_gain(self.conv_nonlinearity.__name__)
            # Weight initialization
            nn.init.orthogonal_(conv.weight, gain=gain)
            # Bias initialization
            nn.init.constant_(conv.bias, 0.0)

In [17]:
a = torch.randn(2, 3)

In [ ]:
a.view()

In [15]:
conv = nn.Conv2d(3, 2, [5, 5])
x = torch.randn(1, 3, 32, 32)
conv(x).shape

torch.Size([1, 2, 28, 28])

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim


import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt


np.random.seed(2)

T = 20
L = 1000
N = 100

x = np.empty((N, L), 'int64')
x[:] = np.array(range(L)) + np.random.randint(-4 * T, 4 * T, N).reshape(N, 1)
data = np.sin(x/T).astype('float32')


class Sequence(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.lstm1 = nn.LSTMCell(1, 51)
        self.lstm2 = nn.LSTMCell(51, 51)
        self.linear = nn.Linear(51, 1)
        
    def forward(self, x, future=0):
        outputs = []
        batch_size, x_size = x.size()
        
        h_t = torch.zeros(batch_size, 51)
        c_t = torch.zeros(batch_size, 51)
        h_t_2 = torch.zeros(batch_size, 51)
        c_t_2 = torch.zeros(batch_size, 51)
        
        chunks = x.chunk(x_size, dim=1)
        
        for x_t in chunks:
            h_t, c_t = self.lstm1(x_t, (h_t, c_t))
            h_t_2, c_t_2 = self.lstm2(h_t, (h_t_2, c_t_2))
            
            output = self.linear(h_t_2)
            
            outputs.append(output)
            
        for _ in range(future):  # if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t_2, c_t_2 = self.lstm2(h_t, (h_t_2, c_t_2))
            
            output = self.linear(h_t_2)
            
            outputs.append(output)
            
        outputs = torch.stack(outputs, dim=1).squeeze(2)
        
        return outputs
        
if __name__ == '__main__':

    np.random.seed(0)
    torch.manual_seed(0)

    input = torch.from_numpy(data[3:, :-1])
    target = torch.from_numpy(data[3:, 1:])
    test_input = torch.from_numpy(data[:3, :-1])
    test_target = torch.from_numpy(data[:3, 1:])

    seq = Sequence()

    criterion = nn.MSELoss()

    # Use LBFGS since we load whole data to train
    optimizer = optim.LBFGS(seq.parameters(), lr=0.8)

    for i in range(2):
        print(f'STEP: {i}')
        def closure():
            optimizer.zero_grad()
            out = seq(input)
            loss = criterion(out, target)
            print(f'Loss: {loss.item()}')
            loss.backward()
            return loss
        optimizer.step(closure)

        # Prediction
        future = 1000
        pred = seq(test_input, future=future)
        loss = criterion(pred[:, :-future], test_target)
        print(f'Test loss: {loss.item()}')
        y = pred.data.numpy()

        # Drawing
        plt.figure(figsize=(30, 10))
        plt.xlabel('x')
        plt.ylabel('y')
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        def draw(yi, color):
            plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth=2.0)
            plt.plot(np.arange(input.size(1), input.size(1)+future), yi[input.size(1):], color + ':', linewidth=2.0)
        draw(y[0], 'r')
        draw(y[1], 'g')
        draw(y[2], 'b')
        
        plt.savefig(f'logs/{i}.pdf')